In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from pathlib import Path
from shapely import MultiPoint, normalize, Point, voronoi_polygons,intersection
import geopandas as gpd
import folium
from itertools import pairwise
from pyroutelib3 import Router, distHaversine
import pyroutelib3.router as rt
from tqdm import tqdm
from random import shuffle
import json
%matplotlib qt

from geovoronoi.plotting import subplot_for_map,plot_voronoi_polys, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords

In [6]:
base_url = 'https://en.wikipedia.org'
station_file_path = Path('./stations.json')
def get_coords(link):
  r = requests.get(base_url + link)
  soup = BeautifulSoup(r.text, 'html.parser')
  coordstring = soup.find(class_="geo").text.strip()
  tuplestring = tuple(map(float, coordstring.split(';')))
  return (tuplestring[1], tuplestring[0])
if station_file_path.exists():
  stations = pd.read_json(station_file_path)
else:
  url = "https://en.wikipedia.org/wiki/List_of_Montreal_Metro_stations"
  stations = pd.read_html(url)[1]
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table')[1]
  links = []
  for tr in table.findAll("tr"):
    trs = tr.findAll("th")
    for each in trs:
      try:
        link = each.find('a')['href']
        links.append(link)
      except:
        pass
  stations['Link']=links[1:]
  stations['coords'] = stations['Link'].apply(get_coords)
  stations.to_json(station_file_path)


In [7]:
stations['coords']

0     [-73.60361, 45.44611]
1     [-73.59306, 45.45139]
2     [-73.58194, 45.45667]
3     [-73.57167, 45.45944]
4     [-73.56694, 45.46278]
              ...          
63      [-73.6237, 45.5305]
64       [-73.62, 45.53528]
65    [-73.60722, 45.54778]
66    [-73.60194, 45.55361]
67        [-73.6, 45.55972]
Name: coords, Length: 68, dtype: object

In [8]:
# router = Router('car', 
#                 'montreal.osm.pbf',
#                 localfileType='pbf')
#router = Router('foot')
router = Router('foot','montreal_canada.osm.pbf',localfileType='pbf')
for coord in stations['coords']:
    router.getArea(coord[1],coord[0])
stations['node'] = list(map(lambda x: router.findNode(x[1],x[0]),stations['coords']))

In [ ]:
stations['node_coord'] = list(map(router.nodeLatLon,stations['node']))

In [ ]:
status, route = router.doRoute(8997083710,6188320842)
list(map(router.nodeLatLon, route))


[(45.524976, -73.5820202), (45.5249321, -73.5820603)]

In [ ]:
router.report()

Loaded 1805162 nodes
Loaded 3983780 foot edges


In [ ]:
canada = gpd.read_file('./lpr_000b16a_e.shp')
canada = canada.to_crs('WGS84')

In [ ]:
quebec = canada[canada['PRENAME'] == 'Quebec']
quebecgeo = quebec.iloc[0]['geometry']
montrealgeo = list(quebecgeo.geoms)[3]


In [ ]:
montrealgeo.contains(Point(-73.5820202,45.524976))

True

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
quebec.plot(ax=ax)
ax.set_xlim(-73.75,-73.472)
ax.set_ylim(45.398,45.623)

for i in range(len(stations)):
  d = stations.iloc[i]
  cds = d['coords']
  ncds = d['node_coord']
  ax.plot(cds[0], cds[1],  marker='o', markersize=8,color='C1', zorder=10)
  ax.plot(ncds[1], ncds[0],  marker='o', markersize=8,color='C2', zorder=10)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
quebec.plot(ax=ax)
ax.set_xlim(-73.75,-73.472)
ax.set_ylim(45.398,45.623)

lats = []
longs = [] 
for n,loc in router.rnodes.items():
  lats.append(loc[0])
  longs.append(loc[1])
  
ax.scatter(longs, lats,  marker='o', s=8,color='C2', zorder=10)

plt.show()

In [ ]:
def total_distance(node1,node2,router):
    status, route = router.doRoute(node1,node2)
    return sum([distHaversine(router.nodeLatLon(p[0]),router.nodeLatLon(p[1])) for p in pairwise(route)]), route
    
def find_station(node,stations,router):
    # Heuristic, find three closest straight line stations
    # and only check these ones.
    dists = {}
    for name,s_node in stations.items():
        dist = distHaversine(router.nodeLatLon(node),router.nodeLatLon(s_node))
        dists[name] = dist
    closest_two = sorted(dists.items(),key=lambda x: x[1])[:2]
    closest_two = list(map(lambda x:x[0],closest_two))
    if dists[closest_two[0]] < dist[closest_two[1]]/2:
        return closest_two[0], router.doRoute(node,stations[closest_two][1])
    dists = {}
    routes = {}
    for name in closest_two:
        d, r = total_distance(node,stations[name],router)
        dists[name] = d
        routes[name] = r
    name = sorted(dists.items(),key=lambda x: x[1])[0][0]
    return name, routes[name]

In [ ]:
s_n_dict = dict(zip(stations['Name'].to_list(),stations['node'].to_list()))


In [ ]:
nodes_file_path = Path("./done_nodes.json")
if nodes_file_path.exists():
    with nodes_file_path.open('r') as f:
        node_stations = json.load(f)
#node_stations = {n : None for n in tqdm(router.rnodes.keys()) if montrealgeo.contains(Point(router.nodeLatLon(n)[1],router.nodeLatLon(n)[0]))}


100%|██████████| 1805162/1805162 [12:18<00:00, 2444.68it/s] 


In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
quebec.plot(ax=ax)
ax.set_xlim(-73.75,-73.472)
ax.set_ylim(45.398,45.623)

lats = []
longs = [] 
for n in node_stations:
  loc = router.nodeLatLon(n)
  lats.append(loc[0])
  longs.append(loc[1])
  
ax.scatter(longs, lats,  marker='o', s=8,color='C2', zorder=10)

plt.show()

In [ ]:
for n in node_stations.keys():
    node_stations[n] = None

In [ ]:
rt.SEARCH_LIMIT = 5000
ns = [n for n,station in node_stations.items() if station is None]
shuffle(ns)
completed = 0
for n in tqdm(ns):
    if node_stations[n] is None:
        # If all neighbouring nodes share the same closest station
        # ngbrs = router.routing[n].keys()
        # ngbrs_stations = [node_stations[ngbr] for ngbr in ngbrs]
        # ngbrs_set = set(ngbrs_stations)

        # if len(set(ngbrs_stations)) == 1 and ngbrs_set != set([None]):
        #     node_stations[n] = ngbrs_stations.pop()
        # else:
            try:
                station, route = find_station(n,s_n_dict,router)
                if route == []:
                    continue
                for n in route:
                    node_stations[n] = station
            except RuntimeError:
                continue
            completed += 1

  4%|▍         | 11124/271062 [20:28<7:58:29,  9.05it/s] 


KeyboardInterrupt: 

In [ ]:
queues = {name:set([]) for name in s_n_dict.keys()}
to_check = {name:node for name,node in s_n_dict.items()}
unvisited = set(node_stations.keys())
while len(unvisited) != 0:
    print(unvisited)
    for name in s_n_dict.keys():
        for node in to_check:
            if node_stations[node] is not None:
                pass
            



{8997083710: None,
 6188320842: 'Mont-Royal',
 8996944085: 'Mont-Royal',
 8996944097: 'Mont-Royal',
 5022472271: 'Mont-Royal',
 8996944106: None,
 26232575: None,
 7450479409: None,
 26232576: None,
 7450479417: None,
 26232577: None,
 510601161: None,
 7450479410: None,
 7450479411: None,
 26232581: 'Guy–Concordia formerly Guy',
 4618656365: None,
 6828696715: None,
 6828696716: None,
 6828696717: None,
 6828696718: None,
 26232619: None,
 6828696719: None,
 26232618: None,
 6828696720: None,
 26232617: None,
 8329284921: None,
 213649537: 'Mont-Royal',
 6767549861: 'Mont-Royal',
 1330277118: 'Mont-Royal',
 26232896: 'Viau',
 26232893: 'Viau',
 1454537655: 'Mont-Royal',
 8997083697: 'Mont-Royal',
 8997083696: 'Mont-Royal',
 2148401275: 'Mont-Royal',
 225104813: 'Mont-Royal',
 3373392916: 'Mont-Royal',
 8997083688: 'Mont-Royal',
 209424717: 'Mont-Royal',
 26233006: None,
 6893619161: 'Berri–UQAM formerly Berri-de Montigny',
 26233012: 'Berri–UQAM formerly Berri-de Montigny',
 903015134

In [ ]:
done_nodes = {key:item for key,item in node_stations.items() if item is not None}

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
quebec.plot(ax=ax,color='gray')
ax.set_xlim(-73.75,-73.472)
ax.set_ylim(45.398,45.623)
from cycler import cycler
dc = cycler(color=['#E8ECFB', '#D9CCE3', '#D1BBD7', '#CAACCB', '#BA8DB4', '#AE76A3', 
                   '#AA6F9E', '#994F88', '#882E72', '#1965B0', '#437DBF', '#5289C7', 
                   '#6195CF', '#7BAFDE', '#4EB265', '#90C987', '#CAE0AB', '#F7F056', 
                   '#F7CB45', '#F6C141', '#F4A736', '#F1932D', '#EE8026', '#E8601C', 
                   '#E65518', '#DC050C', '#A5170E', '#72190E', '#42150A'])
names = set(done_nodes.values())
ax.set_prop_cycle(dc)
for i,name in enumerate(names):
    lats = []
    longs = [] 
    for n in done_nodes.keys():
        if done_nodes[n] == name:
            loc = router.nodeLatLon(n)
            lats.append(loc[0])
            longs.append(loc[1])
        
    ax.scatter(longs, lats,  marker='o', s=8, zorder=10,label=name)

plt.legend()
plt.show()

In [ ]:
import json
with open('./done_nodes.json','w') as f:
    json.dump(node_stations,f)

In [ ]:
router.nodeLatLon(289407299)

(45.5044873, -73.6156014)

In [ ]:
if not quebecgeo.is_valid or quebecgeo.is_empty:
 print('Invalid Geometry')
else:
 print('Geometry Valid')
 
for i, row in stations.iterrows():
  cds = row['coords']
  name = row['Name']
  p = Point(cds[0], cds[1])
  if not quebecgeo.contains(p):
     print(f'{name} ||| OUTSIDE BOUNDARY')

Geometry Valid


In [ ]:
arcords = np.array(list(map(list, stations['coords'].tolist())))
# arcords = np.roll(arcords,1,axis=1)
region_polys, region_pts = voronoi_regions_from_coords(arcords,quebecgeo)
fig, ax = plt.subplots(figsize=(12,12))
plot_voronoi_polys(ax, region_polys)
ax.set_xlim(-73.75,-73.472)
ax.set_ylim(45.398,45.623)

c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\sha

IndexError: too many indices for array: array is 0-dimensional, but 2 were indexed

In [ ]:
region_polys

{0: <MULTIPOLYGON (((-73.591 45.43, -73.592 45.43, -73.592 45.43, -73.592 45.43,...>,
 1: <POLYGON ((-73.704 45.544, -73.704 45.545, -73.704 45.545, -73.703 45.545, -...>,
 2: <MULTIPOLYGON (((-73.593 45.465, -73.58 45.468, -73.579 45.468, -73.58 45.46...>,
 3: <MULTIPOLYGON (((-73.604 45.46, -73.593 45.465, -73.593 45.465, -73.586 45.4...>,
 4: <MULTIPOLYGON (((-73.578 45.469, -73.574 45.467, -73.564 45.454, -73.564 45....>,
 5: <POLYGON ((-73.574 45.467, -73.56 45.466, -73.56 45.466, -73.56 45.465, -73....>,
 6: <POLYGON ((-73.714 45.437, -73.714 45.437, -73.715 45.437, -73.715 45.438, -...>,
 7: <POLYGON ((-73.722 45.438, -73.723 45.438, -73.723 45.438, -73.724 45.439, -...>,
 8: <POLYGON ((-73.635 45.596, -73.653 45.525, -73.654 45.525, -73.667 45.531, -...>,
 9: <POLYGON ((-73.575 45.574, -73.585 45.605, -73.533 45.573, -73.575 45.574))>,
 10: <POLYGON ((-73.545 45.54, -73.563 45.55, -73.539 45.551, -73.528 45.548, -73...>,
 11: <POLYGON ((-73.539 45.551, -73.563 45.566, -73.574 4

In [ ]:
fig,ax = plt.subplots(figsize=(12,12),dpi=300)
station_points = MultiPoint(arcords)
polys = voronoi_polygons(station_points,extend_to=quebecgeo)
region_polys, region_pts = voronoi_regions_from_coords(arcords,quebecgeo)
for p in polys.geoms:
    gpd.GeoSeries(intersection(p,quebecgeo)).plot(ax=ax)
plt.xlim(-73.75,-73.472)
plt.ylim(45.398,45.623)
plt.scatter(arcords[:,0],arcords[:,1],zorder=100,
            marker='o', s=8,color='C1')
plt.show()

c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\Rigel\miniconda3\envs\voronoi\Lib\site-packages\sha

In [ ]:
m = folium.Map(location=[45.50884, -73.58781], zoom_start=13, tiles='CartoDB positron')
for count, r in enumerate(region_polys.values()):
  dfindex = region_pts[count][0]
  d = stations.loc[dfindex]
  name = d['Name']

  popup = f'{name} Station'

  geo_j = folium.GeoJson(data=r, style_function=lambda x: {'color' : 'silver'}, tooltip = name)
  folium.Popup(popup).add_to(geo_j)
  geo_j.add_to(m)

for i in range(len(stations)):
  d = stations.iloc[i]
  cds = d['coords']
  name = d['Name']
  #col = d['COL']

  folium.Circle(
    radius=50,
    location=[cds[1], cds[0]],
    popup = name,
    #color = f'#{col}',
    fill=True,
    tooltip = name).add_to(m)

In [ ]:
m